<h1>This is really about getting the data together, step 2</h1>

In [4]:
import requests
import pandas as pd
import numpy as np


In [40]:
CLIENT_ID = 'BOIJJKYCZMDIUBCUSMDOXANZCVNFNKWTNS3KJZXEB4PLHQHI' # your Foursquare ID
CLIENT_SECRET = '5EJRIL4HGEV0XWVIORAKGQV225IKNKGLULRAKZGPVHB4RXS0' # your Foursquare Secret
VERSION = '20210715' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BOIJJKYCZMDIUBCUSMDOXANZCVNFNKWTNS3KJZXEB4PLHQHI
CLIENT_SECRET:5EJRIL4HGEV0XWVIORAKGQV225IKNKGLULRAKZGPVHB4RXS0


In [6]:
# Get rid of checkins users and vists
# toronto_onehot = toronto_onehot.drop(['NOT FOUND'],axis=1)
#venue_details = venue_details.drop(['CheckinsCount', 'UsersCount', 'VisitsCount'], axis=1)
#venue_details.to_pickle("venue_details.pkl")

In [161]:
# I need to keep adding details to my pickle file and then remember to save again when I'm done
# BE CAREFUL!! tHIS IS READING NOT WRITING!!!!
# venue_details = pd.read_pickle("venue_details.pkl")

In [175]:
venue_details.to_csv("venue_details.csv")

In [22]:
# This is my dataframe for gathering detailed venue data
#venue_details = pd.DataFrame(columns = ["Venue_Id",
#                        "Name",
#                        "TipsCount",
#                        "LikesCount",
#                        "Rating",
#                        "RatingSignals",
#                        "PriceTier"])

In [176]:
venue_details.to_pickle("venue_details.pkl")

In [177]:
def getVenueDetails(venue_id):
    # NOTE: This is a premium call
    #"price": {
    #    "tier": 3
    #    "message": "Expensive"
    #    "currency": "$"
    #    }
    #"stats": {
    #    "checkinsCount": 364591,
    #    "usersCount": 311634,
    #    "tipCount": 1583,
    #    "visitsCount": 854553
    #  },
    #  "url": "http://www.centralparknyc.org",
    #  "likes": {
    #    "count": 17370,
    #    "summary": "17370 Likes" }
    #"rating": 9.8,
    # "ratingSignals"
     # create the API request URL
    
    # Don't run this again if we already have the data, since these are premium calls
    
    if(venue_details['Venue_Id'] == venue_id).any():
        # print("Would be a wasted API call")
        #  FOR CURRENT TESTING I AM DISABLING THIS IMPORTANT CODE - WHICH PREVENTS WASTED REQUESTS *********************
        return "WASTE"
        
    
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
    
    #print(url)
    
    # make the GET request
    result = requests.get(url).json()
    
    if "meta" in result:
        if "errorType" in result["meta"]:
            errorMessage = result["meta"]["errorType"]
            # print("VENUE_ID: " + venue_id + " - WE GOT THIS ERROR MESSAGE: " + errorMessage)
            return "QUOTA"
    else:
        print("meta not in result")
        
    try:
        result = result["response"]['venue']
    except Exception:
        print("Non-Excessive Requests Exception with venue_id: " + venue_id + " - REMOVE VENUE")
        return "ERROR"
    
    stats = result["stats"]
    pricetier = 0
    rating = 0
    ratingSignals = 0
    tipCount = 0
    
    # add ratingSignals
    if "price" in result:
        price = result["price"]
        if "tier" in price:
            pricetier = price["tier"]
    if "tipCount" in stats:
        tipCount = stats["tipCount"]
    if "rating" in result:
        rating = result["rating"]
    if "ratingSignals" in result:
        ratingSignals = result["ratingSignals"]

        
    venue_details.loc[len(venue_details.index)] = [result["id"],
                                                  result["name"],
                                                  tipCount,
                                                  result["likes"]["count"],
                                                  rating,
                                                  ratingSignals,
                                                  pricetier,
                                                  tipCount + (result["likes"]["count"] * 2) + ratingSignals]
    return "SUCCESS"

In [12]:
# I want a number which helps show how popular and fun a place is - giving x2 for LIKES
# df['DataFrame Column'] = df['DataFrame Column'].fillna(0)
#venue_details["TipsCount"] = venue_details["TipsCount"].fillna(0);
#venue_details["LikesCount"] = venue_details["LikesCount"].fillna(0);
#venue_details["RatingSignals"] = venue_details["RatingSignals"].fillna(0);


In [13]:
#venue_details["VoteSum"] = (venue_details["TipsCount"]) + (venue_details["LikesCount"] * 2) + (venue_details["RatingSignals"])

In [14]:
#venue_details.sort_values(by="VoteSum", ascending=False)[0:60]

In [15]:
# Restore a bunch of dataframes

# All the toronto venues
toronto_venues = pd.read_pickle("toronto_venues_data.pkl")
# Only the fun toronto venues
toronto_venues_fun = pd.read_pickle("toronto_venues_fun.pkl")
# Shows the total fun score across neighborhoods
toronto_sum = pd.read_pickle("toronto_most_fun.pkl")
# For NYC, I only have the fun for now
nyc_venues_fun = pd.read_pickle("nyc venues only fun.pkl")
# For NYC, total fun scores across neighborhoods
nyc_sum= pd.read_pickle("nyc_most_fun.pkl")

In [173]:
# Populate venue data by looping through most popular neighborhoods in order of popularity
def loadVenueData(aggdataframe, venuedataframe):
    detailcounter = 0

    for index, row in aggdataframe[0:10].iterrows():
        print("Index: " + str(index))
        neighborhood = aggdataframe.iloc[index]["Neighborhood"]
        print(neighborhood)
        print('------------------------------------------')
        
        for index, row in venuedataframe[venuedataframe["Neighborhood"] == neighborhood].iterrows():
            if detailcounter > 500:
                print("WE HAVE REACHED OUR GOAL")
                return

            # Maybe this iloc approach was wrong!!!
            #venue_id = venuedataframe.iloc[index]["Venue_Id"]
            #neighborhood = venuedataframe.iloc[index]["Neighborhood"]
            #print(venue_id + ", " + neighborhood)
            venue_id = row["Venue_Id"]
            neighborhood = row["Neighborhood"]
            #print(venue_id + ", " + neighborhood)
            
            detailSuccess = getVenueDetails(venue_id)
            # WASTE, QUOTA, ERROR, SUCCESS
            
            if(detailSuccess == "SUCCESS"):
                print(str(detailcounter) + ': ' + venue_id)
                # For now, I am limiting tries even if nothing works - PUT THIS BACK UNDER SUCCESS
                detailcounter +=1
                
            elif detailSuccess == "QUOTA":
                print("WE HAVE REACHED OUR QUOTA")
                return



In [174]:
loadVenueData(toronto_sum, toronto_venues_fun)
#loadVenueData(nyc_sum, nyc_venues_fun)

Index: 0
First Canadian Place, Underground city
------------------------------------------
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
0: 50468014e4b01c18bb731df8
Would be a wasted API call
Would be a wasted

Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
29: 4bacf405f964a520881d3be3

109: 4b2d27b4f964a520a6cf24e3
110: 594ff53c2be42528bcc1bdb7
111: 55457219498ea16d16550d3d
112: 4d9b85a6913a236a733fa108
113: 4b919f56f964a520a4ca33e3
114: 53f5e2fd498ee389ab52b446
115: 4afca145f964a5208e2422e3
116: 4b746c9df964a520c1db2de3
117: 4ad88725f964a5200d1221e3
118: 4ad4c061f964a520abf720e3
119: 4ba6b463f964a520416a39e3
120: 54121f6a498e0aefc0da0ac9
121: 4bedf8b5e24d20a17b567214
122: 4bb942a7cf2fc9b6596ca002
123: 4af36863f964a52053ed21e3
124: 4ab17387f964a520866920e3
125: 4b654efaf964a520aeec2ae3
126: 58a0962975e13747e99b1fa9
127: 52423fb211d22dbd0eea431b
128: 52d4578e498e5273f7099477
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
Would be a wasted API call
129: 5acd155ea879216dfece538d
Would be a wasted API call
Would be a wasted API call
Would be a wasted A

In [150]:
venue_details.shape

(1481, 8)

In [127]:
# For now, try always updating pkl file
venue_details.to_pickle("venue_details.pkl")

In [137]:
# I want to filter down all my data now to the top 10 neighborhoods
top_tor_neigh = toronto_sum["Neighborhood"][0:10]
top_nyc_neigh = nyc_sum["Neighborhood"][0:10]
print(top_tor_neigh)
print(top_nyc_neigh)

0               First Canadian Place, Underground city
1                       Commerce Court, Victoria Hotel
2    Harbourfront East, Union Station, Toronto Islands
3             Toronto Dominion Centre, Design Exchange
4     Stn A PO Boxes, 25 The Esplanade, Enclave of M5E
5                             Richmond, Adelaide, King
6                             Garden District, Ryerson
7                                 Church and Wellesley
8                                       St. James Town
9                                   Central Bay Street
Name: Neighborhood, dtype: object
0           Murray Hill
1            South Side
2               Astoria
3          East Village
4            North Side
5             Yorkville
6          West Village
7       Upper West Side
8    Financial District
9              Downtown
Name: Neighborhood, dtype: object


In [152]:
all_top_venues["Neighborhood"].unique()

array(['Church and Wellesley', 'Garden District, Ryerson',
       'St. James Town', 'Central Bay Street', 'Richmond, Adelaide, King',
       'Harbourfront East, Union Station, Toronto Islands',
       'Toronto Dominion Centre, Design Exchange',
       'Commerce Court, Victoria Hotel',
       'Stn A PO Boxes, 25 The Esplanade, Enclave of M5E',
       'First Canadian Place, Underground city', 'Downtown', 'North Side',
       'South Side', 'Yorkville', 'Upper West Side', 'Murray Hill',
       'East Village', 'West Village', 'Financial District', 'Astoria'],
      dtype=object)

<h1>This is where I limit all the fun venues data to the Top 10 Neighborhoods</h1>

In [140]:
# df[df['col1'].isin(['a', 'c', 'h'])]
all_top_venues = all_venues_fun[all_venues_fun["Neighborhood"].isin(top_tor_neigh) | all_venues_fun["Neighborhood"].isin(top_nyc_neigh)]


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory,City
123,Church and Wellesley,43.66586,-79.38316,5bd2379cbcbf7a0039a2d7b9,Storm Crow Manor,43.666840,-79.381593,Theme Restaurant,Ethnic Food,Toronto
124,Church and Wellesley,43.66586,-79.38316,51a11515498ef97667980b8c,DanceLifeX Centre,43.666956,-79.385297,Dance Studio,Recreation,Toronto
125,Church and Wellesley,43.66586,-79.38316,59ecf741da2e006b9e11a3d3,The Alley,43.665922,-79.385567,Bubble Tea Shop,CoffeeDessert,Toronto
126,Church and Wellesley,43.66586,-79.38316,5d9399e49b61d90008bac7b0,Bar Volo,43.665462,-79.385692,Beer Bar,Bar,Toronto
127,Church and Wellesley,43.66586,-79.38316,4df4456052b100c2d7fdca23,Smith,43.666927,-79.381421,Breakfast Spot,CheapMeal,Toronto
128,Church and Wellesley,43.66586,-79.38316,4fb593c0e4b05a76e2ab2951,Como En Casa,43.665160,-79.384796,Mexican Restaurant,Ethnic Food,Toronto
129,Church and Wellesley,43.66586,-79.38316,4b7af787f964a520f9482fe3,Glad Day Bookshop,43.665271,-79.380785,Bookstore,FunStore,Toronto
130,Church and Wellesley,43.66586,-79.38316,5966b520fc9e94307406dafe,Si Lom,43.665010,-79.380683,Thai Restaurant,Ethnic Food,Toronto
131,Church and Wellesley,43.66586,-79.38316,4caf69ae9b34199c9a2ab763,Fabarnak,43.666377,-79.380964,Restaurant,CheapMeal,Toronto
132,Church and Wellesley,43.66586,-79.38316,534c20fa498ee7bc010834ce,Sansotei Ramen 三草亭,43.666735,-79.385353,Ramen Restaurant,Ethnic Food,Toronto


In [151]:
all_top_venues[0:50]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory,City
123,Church and Wellesley,43.66586,-79.38316,5bd2379cbcbf7a0039a2d7b9,Storm Crow Manor,43.666840,-79.381593,Theme Restaurant,Ethnic Food,Toronto
124,Church and Wellesley,43.66586,-79.38316,51a11515498ef97667980b8c,DanceLifeX Centre,43.666956,-79.385297,Dance Studio,Recreation,Toronto
125,Church and Wellesley,43.66586,-79.38316,59ecf741da2e006b9e11a3d3,The Alley,43.665922,-79.385567,Bubble Tea Shop,CoffeeDessert,Toronto
126,Church and Wellesley,43.66586,-79.38316,5d9399e49b61d90008bac7b0,Bar Volo,43.665462,-79.385692,Beer Bar,Bar,Toronto
127,Church and Wellesley,43.66586,-79.38316,4df4456052b100c2d7fdca23,Smith,43.666927,-79.381421,Breakfast Spot,CheapMeal,Toronto
128,Church and Wellesley,43.66586,-79.38316,4fb593c0e4b05a76e2ab2951,Como En Casa,43.665160,-79.384796,Mexican Restaurant,Ethnic Food,Toronto
129,Church and Wellesley,43.66586,-79.38316,4b7af787f964a520f9482fe3,Glad Day Bookshop,43.665271,-79.380785,Bookstore,FunStore,Toronto
130,Church and Wellesley,43.66586,-79.38316,5966b520fc9e94307406dafe,Si Lom,43.665010,-79.380683,Thai Restaurant,Ethnic Food,Toronto
131,Church and Wellesley,43.66586,-79.38316,4caf69ae9b34199c9a2ab763,Fabarnak,43.666377,-79.380964,Restaurant,CheapMeal,Toronto
132,Church and Wellesley,43.66586,-79.38316,534c20fa498ee7bc010834ce,Sansotei Ramen 三草亭,43.666735,-79.385353,Ramen Restaurant,Ethnic Food,Toronto


In [141]:
# 8-16-2021 - all_top_venues is all the nyc and tor venue data filtered to the top 10 neighs in each city
all_top_venues.to_pickle("all_top_venues.pkl")

In [65]:
# ADD CITY COLUMNS TO EACH OF THE CITY DATAFRAMES
toronto_venues_fun['City'] = 'Toronto'
nyc_venues_fun['City'] = 'New York'

In [108]:
#Combine the venues data from toronto and nyc so I can merge the venue_details
all_venues_fun = toronto_venues_fun.append(nyc_venues_fun, ignore_index=True)
all_venues_fun.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory,City
0,Lawrence Park,43.728020,-79.388790,50e6da19e4b0d8a78a0e9794,Lawrence Park Ravine,43.726963,-79.394382,Park,Recreation,Toronto
1,Lawrence Park,43.728020,-79.388790,5082ef77e4b0a7491cf7b022,Zodiac Swim School,43.728532,-79.382860,Swim School,Recreation,Toronto
2,Davisville North,43.712751,-79.390197,4ba011c2f964a5204a5737e3,Sherwood Park,43.716551,-79.387776,Park,Recreation,Toronto
3,Davisville North,43.712751,-79.390197,4adb2fd3f964a520c42421e3,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot,CheapMeal,Toronto
4,Davisville North,43.712751,-79.390197,4b0b3691f964a520c62e23e3,Subway,43.708474,-79.390674,Sandwich Place,CheapMeal,Toronto


In [109]:
all_venues_fun.to_pickle("all_venues_fun.pkl")

<h1>Merge all venues with all venue details now
BUT - I think this should be focused on the top 10 neighborhoods for Tor and Nyc!</h1>

In [130]:

all_venues_and_details = all_venues_fun.merge(venue_details,how="left", on="Venue_Id")

In [135]:
all_venues_and_details[1000:1040] #[1370:1415]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory,City,Name,TipsCount,LikesCount,Rating,RatingSignals,PriceTier,VoteSum
1000,"First Canadian Place, Underground city",43.648429,-79.38228,51ae5eaa498e5c629f21f431,Olly Fresco's,43.646912,-79.379597,Deli / Bodega,CheapMeal,Toronto,Olly Fresco's,4.0,18.0,7.8,23.0,1,63.0
1001,"First Canadian Place, Underground city",43.648429,-79.38228,5a4fdf56772fbc5e9fa73c7f,Chotto Matte,43.646473,-79.378782,Japanese Restaurant,Ethnic Food,Toronto,Chotto Matte,2.0,13.0,8.2,16.0,2,44.0
1002,"First Canadian Place, Underground city",43.648429,-79.38228,5214e7c111d2a83379eae21f,The Chase,43.650952,-79.379422,New American Restaurant,Fancy Food,Toronto,The Chase,59.0,145.0,8.2,212.0,3,561.0
1003,"First Canadian Place, Underground city",43.648429,-79.38228,4ad4c05ef964a520d8f620e3,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum,Entertainment,Toronto,Hockey Hall Of Fame (Hockey Hall of Fame),79.0,407.0,8.7,501.0,NaN,1394.0
1004,"First Canadian Place, Underground city",43.648429,-79.38228,4e048b98315168be7fd7ee4d,Estiatorio Volos,43.650329,-79.384533,Greek Restaurant,Ethnic Food,Toronto,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1005,"First Canadian Place, Underground city",43.648429,-79.38228,56df0683498e7d0423e91b5b,iQ Food Co. (First Canadian Place),43.648357,-79.382192,Salad Place,Ethnic Food,Toronto,iQ Food Co. (First Canadian Place),1.0,7.0,7.4,9.0,1,24.0
1006,"First Canadian Place, Underground city",43.648429,-79.38228,4b5ca7d8f964a5207c3c29e3,Beerbistro,43.649419,-79.377237,Gastropub,Bar,Toronto,Beerbistro,128.0,323.0,8.6,478.0,2,1252.0
1007,"First Canadian Place, Underground city",43.648429,-79.38228,4ba90db8f964a5203f073ae3,Sweet Lulu,43.650557,-79.381175,Asian Restaurant,Ethnic Food,Toronto,Sweet Lulu,11.0,17.0,7.6,28.0,2,73.0
1008,"First Canadian Place, Underground city",43.648429,-79.38228,563d2f2dcd10bcf27ae37c3b,Pilot Coffee Roasters,43.645018,-79.380415,Coffee Shop,CoffeeDessert,Toronto,Pilot Coffee Roasters,12.0,50.0,8.4,69.0,1,181.0
1009,"First Canadian Place, Underground city",43.648429,-79.38228,5a5a59c5a423620ec1dafd41,Pi Co.,43.648651,-79.385874,Pizza Place,Ethnic Food,Toronto,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<H1>This is where I create the concept of a general Score for each venue</H1>

In [118]:
#all_venues_and_details.sort_values(by="VoteSum", ascending=False)[0:50]
# all_venues_and_details["Score"] = all_venues_and_details["VoteSum"] * all_venues_and_details["Rating"]
all_venues_and_details.sort_values(by="Score", ascending=False)[50:100]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory,City,Name,TipsCount,LikesCount,Rating,RatingSignals,PriceTier,VoteSum,Score
5606,West Village,40.734434,-74.006180,543c7c63498e0a3393da47a2,Via Carota,40.733052,-74.003573,Italian Restaurant,Ethnic Food,New York,Via Carota,161.0,636.0,9.3,812.0,3,2245.0,20878.5
4765,Upper West Side,40.787658,-73.977059,3fd66200f964a5203fea1ee3,Good Enough to Eat,40.785979,-73.972764,Breakfast Spot,CheapMeal,New York,Good Enough to Eat,314.0,645.0,7.8,1058.0,2,2662.0,20763.6
5603,West Village,40.734434,-74.006180,45379ccef964a520c13b1fe3,The Little Owl,40.732441,-74.005424,American Restaurant,Ethnic Food,New York,The Little Owl,248.0,611.0,8.8,888.0,3,2358.0,20750.4
5573,Soho,40.722184,-74.000657,49bd4e25f964a52067541fe3,Equinox SoHo,40.723973,-73.997042,Gym,Recreation,New York,Equinox SoHo,129.0,636.0,8.9,799.0,0,2200.0,19580.0
6939,Steinway,40.775923,-73.902290,49c68da1f964a5205b571fe3,Martha's Country Bakery,40.773924,-73.907555,Bakery,CoffeeDessert,New York,Martha's Country Bakery,160.0,477.0,9.0,646.0,2,1760.0,15840.0
5580,Soho,40.722184,-74.000657,49cebdeaf964a520815a1fe3,Macao Trading Co.,40.719759,-74.003981,Asian Restaurant,Ethnic Food,New York,Macao Trading Co.,227.0,258.0,9.0,1002.0,3,1745.0,15705.0
6935,Steinway,40.775923,-73.902290,4f68de6bd5fbee32e5f4f3a5,SingleCut Beersmiths,40.778387,-73.901902,Brewery,Bar,New York,SingleCut Beersmiths,95.0,484.0,9.3,593.0,1,1656.0,15400.8
5612,West Village,40.734434,-74.006180,4a552b26f964a520c4b31fe3,Malatesta Trattoria,40.732808,-74.008522,Italian Restaurant,Ethnic Food,New York,Malatesta Trattoria,200.0,445.0,8.7,666.0,3,1756.0,15277.2
4742,Upper West Side,40.787658,-73.977059,4d68839467a8f04df7ee01be,Osteria Cotta,40.785578,-73.972836,Italian Restaurant,Ethnic Food,New York,Osteria Cotta,172.0,442.0,8.5,671.0,2,1727.0,14679.5
206,"Regent Park, Harbourfront",43.654260,-79.360636,51ddecee498e1ffd34185d2f,El Catrin,43.650601,-79.358920,Mexican Restaurant,Ethnic Food,Toronto,El Catrin,162.0,470.0,8.1,707.0,3,1809.0,14652.9


In [117]:
all_venues_and_details.to_pickle("all_venues_and_details.pkl")

In [70]:
toronto_venues_fun.shape #1622
#nyc_venues_fun.shape # 7495

(1622, 10)

<H1>This is where I added the City column into the venue details</H1>

In [53]:
# Combine whatever data we have in the venue details with what we have in toronto venues
# MERGE IN TORONTO
#city_fun = venue_details.merge(toronto_venues_fun, how='left', on='Venue_Id',left_index=True)

# MERGE IN NYC
city_fun = venue_details.merge(nyc_venues_fun, how='left', on='Venue_Id',left_index=True)

In [64]:
# BE CAREFUL WITH RUNNING THIS
# city_fun['City'] = 'Toronto'
#city_fun['City'] = 'Toronto'
city_fun[city_fun['Neighborhood'].isnull()==False]
#city_fun.head(30)

,Venue_Id,Name,TipsCount,LikesCount,Rating,RatingSignals,PriceTier,VoteSum,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory
6007.0,4d3e1be3ae942d43e09e9f9a,El Pequeno Coffee Shop,14,9,6.1,23,2,55,Jackson Heights,40.751981,-73.882821,El Pequeno Coffee Shop,40.748083,-73.880205,Latin American Restaurant,Ethnic Food
6009.0,5e44505ffdc6610008c7d741,Thaan,1,1,0.0,0,2,3,Jackson Heights,40.751981,-73.882821,Thaan,40.749849,-73.881865,Thai Restaurant,Ethnic Food
6010.0,4c671600aebea593c63075d0,Casa Colombia,3,4,5.8,9,2,20,Jackson Heights,40.751981,-73.882821,Casa Colombia,40.748263,-73.879485,South American Restaurant,Ethnic Food
6012.0,4b9a8df9f964a520acc035e3,La Gata Golosa Bakery,7,8,5.6,17,1,40,Jackson Heights,40.751981,-73.882821,La Gata Golosa Bakery,40.750458,-73.877701,South American Restaurant,Ethnic Food
6013.0,52f693e6498ed9d2a3fbad75,Eim Khao Mun Kai Elmhurst อิ่ม ข้าวมันไก่เอ็มเ...,26,99,8.7,128,1,352,Elmhurst,40.744049,-73.881656,Eim Khao Mun Kai Elmhurst อิ่ม ข้าวมันไก่เอ็มเ...,40.743014,-73.883057,Thai Restaurant,Ethnic Food
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9907.0,4bbe631782a2ef3b764b2bd2,Ralph's Famous Italian Ices,10,13,8.2,22,1,58,Middle Village,40.716415,-73.881143,Ralph's Famous Italian Ices,40.712856,-73.879132,Dessert Shop,CoffeeDessert
9909.0,4bd60bf4cfa7b713faec26da,Toyo,11,17,7.7,28,2,73,Middle Village,40.716415,-73.881143,Toyo,40.712611,-73.878948,Sushi Restaurant,Ethnic Food
9911.0,53372ea7498ed5a1955d86ef,Rico's Chicken,2,11,7.3,13,2,37,Middle Village,40.716415,-73.881143,Rico's Chicken,40.713009,-73.877336,South American Restaurant,Ethnic Food
9914.0,599b1f75ca18ea189fe0dfff,The Salvation Army Family Store & Donation Center,0,0,6.5,0,0,0,Middle Village,40.716415,-73.881143,The Salvation Army Family Store & Donation Center,40.712672,-73.878501,Thrift / Vintage Store,FunStore


In [52]:
city_fun.tail()

,Venue_Id,Name,TipsCount,LikesCount,Rating,RatingSignals,PriceTier,VoteSum,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory,City
NaN,4bbe631782a2ef3b764b2bd2,Ralph's Famous Italian Ices,10,13,8.2,22,1,58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Toronto
NaN,4bd60bf4cfa7b713faec26da,Toyo,11,17,7.7,28,2,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Toronto
NaN,53372ea7498ed5a1955d86ef,Rico's Chicken,2,11,7.3,13,2,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Toronto
NaN,599b1f75ca18ea189fe0dfff,The Salvation Army Family Store & Donation Center,0,0,6.5,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Toronto
NaN,4bc63c3651b376b095391b6f,SUBWAY,7,1,6.3,6,1,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Toronto


In [45]:
# To remove rows by index
# venue_details = venue_details.drop(index=[12,13])
venue_details.tail(40)

,Venue_Id,Name,TipsCount,LikesCount,Rating,RatingSignals,PriceTier,VoteSum
559,4b7ff72ff964a520684730e3,Forest Hills Bagels,24,86,9.0,113,1,309
560,4c377a13ae2da59377f4fec5,Yellowstone Park,4,21,7.9,26,0,72
561,51c3a999498e5db561b5346a,iLoveKickboxing,21,6,8.9,27,0,60
562,4b97f9ebf964a520822335e3,MacDonald Park,8,31,8.1,41,0,111
563,46b1ef0df964a520a0491fe3,Bangkok Cuisine,34,62,8.5,93,1,251
564,4b7d8e1af964a5205dc62fe3,bambooYOGA,11,13,8.1,18,0,55
565,4bb6860ef562ef3b4c513097,Empire Wines & Liquors,6,25,8.0,35,0,91
566,54568cc5498ed42af61bc55d,Tu Casa Restaurant,4,10,7.9,15,2,39
567,4c64b110ee03ef3b735dbaac,Zen and Yoga,2,14,8.0,17,0,47
568,4f6cb7a0e4b0a6b770c9b395,Gloria Pizza,20,39,7.9,65,1,163


In [34]:
nyc_sum.head()
# Murray Hill is #1

,index,Neighborhood,Bar,CheapMeal,CoffeeDessert,Entertainment,Ethnic Food,Fancy Food,FunStore,Recreation,TotalFun
0,184,Murray Hill,11,19,14,2,59,5,0,13,123
1,252,South Side,22,10,9,2,35,3,1,8,90
2,6,Astoria,14,13,13,0,38,4,0,7,89
3,81,East Village,20,7,11,2,40,1,5,3,89
4,194,North Side,17,7,21,2,29,2,2,6,86


In [46]:
nyc_venues_fun[(nyc_venues_fun["Neighborhood"] == "Clinton") & (nyc_venues_fun["SimpleCategory"] == "Entertainment")]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory
4342,Clinton,40.759101,-73.996119,4b6a09cff964a5206ac32be3,Pershing Square Signature Theater,40.759228,-73.995232,Theater,Entertainment
4344,Clinton,40.759101,-73.996119,4c379ecf1e06d13a4e49763e,Playwrights Horizons,40.758818,-73.993574,Theater,Entertainment
4346,Clinton,40.759101,-73.996119,5a235575e179100e0425abc7,UCB Theatre Hell’s Kitchen,40.760500,-73.997863,Comedy Club,Entertainment
4358,Clinton,40.759101,-73.996119,5711b92e498e677b825cc78d,Yotel - Rooftop Cinema Club,40.759429,-73.995047,Movie Theater,Entertainment
4359,Clinton,40.759101,-73.996119,4b6cdf69f964a520af5a2ce3,The Actors Studio,40.760339,-73.993280,Theater,Entertainment
4367,Clinton,40.759101,-73.996119,4a0b4df9f964a520e8741fe3,Baryshnikov Arts Center,40.756283,-73.997181,Theater,Entertainment
4373,Clinton,40.759101,-73.996119,4ab424b4f964a520117020e3,The Westside Theater,40.759350,-73.992765,Indie Theater,Entertainment
4388,Clinton,40.759101,-73.996119,4fa709c3e4b047f0ed7001d7,The Alice Griffin Jewel Box Theatre,40.759463,-73.995185,Theater,Entertainment
4392,Clinton,40.759101,-73.996119,58a3c4cce9dad156c1d43d93,Green Room 42,40.759448,-73.995083,Performing Arts Venue,Entertainment
